In [1]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import itertools
import DeepWEST 
import os

#### Load Data ( .prmtop and .nc should be present)

In [2]:
data_dir = "/home/aaojha/Downloads/DeepWEST/data_for_DeepWEST/alanine_dipeptide"
traj_file = os.path.join(data_dir, "no_bias_solvent.nc")
top = os.path.join(data_dir, "system.prmtop")

In [3]:
heavy_atoms_file = os.path.join("heavy_atoms_md.txt")
phi_psi_file = os.path.join("phi_psi_md.txt")

In [ ]:
DeepWEST.create_heavy_atom_xyz_solvent(traj = traj_file, top = top, 
                                    heavy_atoms_array = heavy_atoms_file, 
                                    start = 0, stop = 25000, stride = 1)

DeepWEST.create_phi_psi_solvent(traj = traj_file, top = top, phi_psi_txt = phi_psi_file, 
                             start = 0, stop = 25000, stride = 1)

In [ ]:
traj_whole = np.loadtxt(heavy_atoms_file)
print(traj_whole.shape)
dihedral = np.loadtxt(phi_psi_file)
print(dihedral.shape)

## EM Clustering Algorithm

In [ ]:
traj_data = traj_whole # data to be used as input to GMM, can be with dihedrals or without dihedrals

In [ ]:
tsne_dims = DeepWEST.tsne_visualize(traj_data)

In [ ]:
# K-Means Clustering and Gaussian Mixture Model
num_clusters = DeepWEST.experiment_with_k_means(traj_data, tsne_dims)
posterior_probabs = DeepWEST.gmm(traj_data, num_clusters)

In [ ]:
indices, labels = DeepWEST.get_clustered_indices(traj_data, posterior_probabs, 
                                              num_traj_indices = 50)
print(indices.shape[0])
np.savetxt("indices_dihed.txt", indices)

In [ ]:
fig, ax = plt.subplots()
for i in range(num_clusters[0][0]+2):
    coor_train = np.where(labels == i)[0]
    ax.scatter(dihedral[coor_train,0], dihedral[coor_train,1], s=5, label=i)
ax.legend()
plt.axes = [[-np.pi, np.pi],[-np.pi, np.pi]]
plt.savefig(os.path.join(data_dir, "RC.jpeg"))
plt.show()

In [ ]:
def print_states_pie_chart():
    labels_final = np.argmax(posterior_probabs, axis = 1)
    coors = np.bincount(labels_final)
    fig1, ax1 = plt.subplots()
    ax1.pie(np.array(coors), autopct='%1.2f%%', startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    print('States population: '+ str(np.array(coors)/len(labels_final)*100)+'%')
    plt.savefig(os.path.join(data_dir, "population.jpeg"))
    plt.show()
print_states_pie_chart()